In [1]:
import json
import spacy
nlp = spacy.load("output/model-best")

: 

: 

In [ ]:
with open('./texts_for_testing/1880_90030.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['plain_text']
spacy_doc = nlp(opinion_of_text)

In [ ]:
spacy_doc.cats